# The GPU Memory Hierarchy - by analogy

Modern server GPUs have a memory hierarchy - a series of storage options that get closer and closer to the compute cores.

The overarching philosophy of using GPUs efficiently is that when we move data closer to the compute cores, we should do as much work as possible on that data before moving it back. This is because the closer we get to the cores, the faster the memory bandwidth becomes. However, the storage options close to the core are small and can only store limited data - in some cases only 256KB vs 80GB in the storage furthest from the core. Our job is to efficiently move data up the memory hierarchy toward the compute cores so that the compute cores are active as much of the time as possible.

In fact, virtually every software choice you can make to make GPUs perform better comes down to memory - efficiently moving model weights, inputs and hidden states around the GPU.

There's only really one choice you can make that affects the actual compute - structuring your model computation as being heavily weighted towards matrix multiplication, which can use the tensor cores on the GPU. One real world example of this choice is in section 3.1 of the [FlashAttention 2 paper](https://arxiv.org/pdf/2307.08691.pdf), where they remove some element wise operations from the inner loop of the algorithm in order to maximize the matrix multiplication FLOPs.

While CUDA exposes both tensor core and cuda core level GEMM that manages memory highly efficiently, it's interesting to think about how to design this type of GEMM using lower level CUDA functions. I highly recommend [this article](https://siboehm.com/articles/22/CUDA-MMM) which discusses this problem in detail.

Moreover, understanding this memory hierarchy is really important in highly practical settings. For example, during LLM inference the KV caching puts a high load on memory bandwidth and massively reduces the batch size compared to the training setting. This is a long way removed from writing CUDA code, but relies on the very same principles.



## Illustrating the memory hierarchy with shoe pairs

Imagine the following scenario - we live in a village on the main island. We have 1000 left shoes, and 1000 right shoes on our island. We want to find all shoe pairs through brute force comparison between different left and right shoes.

The only way to find a pair is by pairwise comparison, and the only person who can do the comparison is a shoe expert who lives at the end of a small island chain.

In this chain, there's one medium sized island that is connected to the main island by a long bridge. This island can store up to 200 shoes, and takes 9 minutes to travel to from the mainland, carrying 1 left shoe and 1 right shoe (or 2 left shoes, or 2 right shoes).

After the medium island is a small island connected to the medium island by a short bridge. This is where the shoe expert lives, and it can store 20 shoes. It takes 1 minute to cross from the medium island to the small island, carrying 1 left shoe and 1 right shoe (or 2 left shoes, or 2 right shoes). So the total village to small island travel time is 10 minutes for 1 left shoe and 1 right shoe.

Assume that the shoe expert takes no time to do comparisons. Also assume that returning shoes from one island to another takes no time. The islands are analogous to the storage options on a GPU, and the expert is analogous to the compute cores. The assumptions we make here are pretty much true on the GPU - data is copied from memory so can be deleted rather than returned higher up in memory, and compute is much faster than memory per byte moved (plus we always do the same amount of compute).

If you were to compare this to an A100, the main island might represent HBM, the medium island L1 cache/SMEM and the small island the registers.

See the diagram below:

<img src="files/Islands.PNG" alt="Islands" width="800"/>

## Maximizing computation at the small island
In this situation, we need to do $1000*1000=10^6$ pairwise comparisons to find all pairs by brute force.

The simplest possible algorithm would be to take 1 left shoe and 1 right shoe at a time to the expert, then return to the main island, for a 10 minute trip each time.

All comparisons in this case would take $10 \text{ (trip time)}*10^6\text{ (number of trips)}=10^7$ minutes

One simple improvement to this would be to fill up the small island with candidate pairs, doing all possible computations on those shoes at the small island.
When we take 20 shoes to the expert, taking $100$ total trip travel minutes, there are 10 left shoes and 10 right shoes. So we can do $10*10=100$ comparisons on each trip. This results in total time being $100 \text{ (trip time)}*10^4\text{ (number of trips)}=10^6$ minutes. 

By doing all possible computation at the small island on each trip, we achieve a 10x increase in speed. This is the first illustration of the principle of doing all possible computation on data we move up the memory hierarchy towards the compute cores.

## Using the medium island

However, we're not using the medium island effectively yet. We know we can do $100$ computations each time at the small island, but if we fill the medium island we could do $10,000$ computations on those shoes before taking them back to the village.

So imagine this algorithm:
- split the left shoes and right shoes into 10 large buckets of 100 shoes each.
- First, take left large bucket 1 and right large bucket 1 to the medium island (this takes $9*100=900$ minutes as it's a total of 200 shoes )
- Then, split those large buckets into 10 further smaller buckets of 10 shoes each.
- Take left small bucket 1 and right small bucket 1 to the small island and do all 100 computations (this takes $1*10=10$ minutes as it's a total of 20 shoes )
- Take the small buckets back to the medium island (call this step X)
- Take left small bucket 1 and right small bucket 2 to the small island and do all 100 computations
- ...
- repeat until all small bucket pairs on medium island exhausted
- Take the large buckets back to the village (call this step Y)
- Take left large bucket 1 and right large bucket 2 to the medium island
- Do all 10,000 computations as in the above loop using smaller buckets
- take the large buckets back
- Take left large bucket 1 and right large bucket 3 to the medium island
- ...
- repeat until all large bucket pairs exhausted

The total run-time of this is a little more complicated.

We have $10*10=100$ large bucket pairs. Travel time for each of these pairs is $900$ minutes, for an overall total of $9*10^4$ minutes total moving large buckets between the village and the medium island

For each large bucket pair, there are $10*10=100$ small bucket pairs. Travel time for each of these pairs is $10$ minutes. This is a total of $100 *100 *10=10^5$ minutes moving small buckets between the medium island and the small island.

This is a total of $1.9*10^5$ minutes, another 5x increase over our previous algorithm.

## Preventing redundant returns
We're not quite done yet - there's still some waste above. I labelled step X and Y above for a reason.

First think about step Y - in it we return left small bucket 1 and a right small bucket 1 from the small island to the medium island as we've just done all those pairwise comparisons.

Directly after that, we take left small bucket 1 back to the small island! This is a waste. We should leave left small bucket 1 at the small island until we've compared all 10 right small buckets on the medium island with it.

Similarly in step Y, we should leave left large bucket 1 at the medium island until we've exhausted all 10 right large buckets we could compare it to. This minimizes movement across the memory hierarchy.

Let's calculate the time here.

10 left large bucket pairs. For each one we move it to medium island, then move 10 right bucket pairs to the medium island.

Each large bucket movement takes 450 minutes (moving two large buckets takes 900 minutes remember)

Total time is $10 \text{ (for each left large bucket)} * (450 \text{ (move the left large bucket)} + 10 * 450 \text{ (move 10 right large buckets)})= 49500 $ minutes to move all large buckets from the village to the medium island. This is around a 2x improvement over the above.

Now for the small buckets it takes 5 minutes to move to the small island (10 minutes for 2 small buckets)

So we have total time is $100 \text{ (for each large bucket pair)}*10 \text{ (for each left small bucket)} * (5 \text{ (move the left small bucket)} + 10 * 5 \text{ (move 10 right small buckets)})= 55000 $ minutes to move all small buckets from the medium island to the small island.

This gives a total of $49500+55000=104500=1.045*10^5$ minutes which is just under a 2x increase on the previous algorithm.

## Conclusion

Overall, we've achieved a 100x speed improvement by efficiently moving the shoes and doing maximum computation for stage in the island chain. Remember that in every algorithm we're doing the same number of calculations - $10^6$ - this is purely about moving the shoes in a better way between the islands.

 This illustrates the memory hierarchy on the GPU using a simple yet practical analogy.